In [1]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy import spatial as sp_spatial
import numba
import os
root_folder = os.getcwd()

In [2]:
# v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "Oblate_Coarse.off"))
#v1, f1 = igl.read_triangle_mesh(os.path.join(root_folder, "Oblate_Finer.off"))
### Parameters for running computation
global Kb
global Kv
global Ka
global gamma
global KbT
global delT
global Kal
Kal=1
Ka=2
Kv=1
Kb=0.01
H0=0
Volume_t= 0.90* 3.14 * 4 / 3
Area_t=4*3.14
KbT=0.001
charTimeStep=0.1
isAdaptiveStep=True
#hdt=0.5*dt
iterations = 5000
outfrequency = 50
tolerance = 1e-3
maxError = 7000

#v_init,f=igl.upsample(v1,f1,1)
gamma=0.1#*(len(v1)/len(v_init))

#max(v1[:,2])
rp=0.161 ###radius of nanoparticle
####center of radious 
x=1.161
y=0
z=0
rho=0.1 ### Potential Range Distance
u=2.4
U=(Kb*u)/((rp)**2)

###rp diameter of particle


In [3]:
import meshzoo
import optimesh
points,cells = meshzoo.tetra_sphere(30)

class Sphere:
    def f(self, x):
        return 1.0 - (x[0] ** 2 + x[1] ** 2 + x[2] ** 2)

    def grad(self, x):
        return -2 * x
v_init, f= optimesh.optimize_points_cells(
    points,cells,"CVT (full)",1.0e-2,100,verbose=False,implicit_surface=Sphere(),
    # step_filename_format="out{:03d}.vtk"
    )

rc=igl.avg_edge_length(v_init,f)*1.72 ###Cut-off Distance 

In [4]:
###Energy Calculations /// Area+Bending+Volume
# @numba.jit
def Energy_area(v,f,Area_t):
    Area_new=cal_areatot(v,f)
    Energy_Area=Ka*((Area_new-Area_t)**2)/Area_t
    return Energy_Area
# @numba.jit
def Energy_volume(v,f,Volume_t):
    volume_new=cal_volumetot(v,f)
    Energy_volume=Kv*((volume_new-Volume_t)**2)/Volume_t
    return Energy_volume  

In [5]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers
def cal_areatot(v,f):
    dbl_area = igl.doublearea(v, f)    
    Areatot = np.sum(dbl_area)/2
    return Areatot
@numba.jit
def cal_volumetot(v,f):
    Volumetot = 0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volumetot+=sum
    return Volumetot

def areaGrad(v,f):
    #n=igl.per_vertex_normals(v,f) ## not using per_vertex_normals for areaGrad direction
    l = igl.cotmatrix(v, f) ###laplacian-operator
    ag = -l.dot(v)
    return ag
def volGrad(v,f):
    npv = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,npv)
    dbl_area = igl.doublearea(v, f)
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]/2
            FaceNorm=face_normal[k]
            vol_ij += (1/3)*Area*FaceNorm
        volumegrad.append(vol_ij)
    return np.array(volumegrad)
def ver_new(v,f):
    adjacent_faces=adjacent_face(v,f)
    Area=igl.doublearea(v,f)
    n = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,n)
    v_b=igl.barycenter(v,f)
    v_new=[]
    for i in range(len(v)):
        face_area=Area[adjacent_faces[i]]
        face_area_sum=np.sum(face_area)
        v_centroid=v_b[adjacent_faces[i]]
        sum_of_area_centroid=np.dot(face_area,v_centroid)
        v_avg=sum_of_area_centroid/face_area_sum
        fnorm=face_normal[adjacent_faces[i]]
        fsum=np.sum(fnorm,axis=0)
        lamda=(np.dot(v_avg,fsum)-np.dot(v[i],fsum))/np.dot(fsum,fsum)
        v_new.append(v_avg-lamda*fsum)    
    return np.array(v_new)

In [6]:
###Force  and Energy Calculations from bending

def Force_Bending(v,f):
    npv = igl.per_vertex_normals(v, f)
    K = igl.gaussian_curvature(v, f)
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    area_voronoi=m.diagonal()
    minv = sp.sparse.diags(1 / m.diagonal())
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    #if (min(sign_H)<0):
    #    print('H_mean changes sign')
    H_mean_signed = H_mean*sign_H
    Lap_H = minv.dot(l.dot(H_mean_signed-H0))
    kn = minv.dot(K)
    first_term = 2*(H_mean_signed-H0)*(H_mean_signed**2 + H0*H_mean_signed -kn)
    totalforce = first_term + Lap_H
    #for i in range (len(v)):
    Force_bending = 2*Kb*npv*totalforce[:,None]*area_voronoi[:,None]
    Eb = 2*Kb*(((H_mean_signed-H0)**2))*area_voronoi
    total_EB = np.sum(Eb)
    
    return Force_bending,total_EB

In [7]:
###Force from Area Constraints
@numba.jit
def Force_Area(Area_t,grad_Area,Area_new):
    Force_Area=-2*(Ka)*((Area_new-Area_t)/Area_t)*grad_Area
    ## Area constraint is not the same as in mem3dg
    return Force_Area

In [8]:
###Force from Volume Area Constraints
@numba.jit
def Force_Volume(Volume_t,grad_Volume,volume_new):
    Force_Volume=-2*(Kv)*((volume_new-Volume_t)/Volume_t)*grad_Volume 
    ## Volume constraint is not the same as in mem3dg
    return Force_Volume

In [9]:
tol=1e-20
def Force_Adhersive(v, f, x, y, z, rho, U, AG):
    distance = np.zeros(len(v))
    coef = np.zeros(len(v))
    dc = np.zeros(len(v))
    coefficinet_derivative_x = np.zeros(len(v))
    coefficinet_derivative_y = np.zeros(len(v))
    coefficinet_derivative_z = np.zeros(len(v))

    for i in range(len(v)):
        distance[i] = (np.sqrt((((v[i][0])-x)**2) +
                       (((v[i][1])-y)**2)+(((v[i][2])-z)**2)))
        dc[i] = distance[i]-(rp)
    
        coef[i] = U*(np.exp(-(2*dc[i])/rho) - 2*np.exp(-dc[i]/rho))
        if abs(coef[i]) < tol or abs(dc[i])>rc:
            coef[i] = 0

        coefficinet_derivative_x[i] = (
            U/(distance[i]*rho))*(-np.exp(-(2*dc[i])/rho) + 2*np.exp(-dc[i]/rho)) * 2 * (v[i][0]-x)
        coefficinet_derivative_y[i] = (
            U/(distance[i]*rho))*(-np.exp(-(2*dc[i])/rho) + 2*np.exp(-dc[i]/rho)) * 2 * (v[i][1]-y)
        coefficinet_derivative_z[i] = (
            U/(distance[i]*rho))*(-np.exp(-(2*dc[i])/rho) + 2*np.exp(-dc[i]/rho)) * 2 * (v[i][2]-z)
        if abs(coefficinet_derivative_x[i]) < tol or abs(dc[i])> rc:
            coefficinet_derivative_x[i] = 0
        if abs(coefficinet_derivative_y[i]) < tol or abs(dc[i])>rc:
            coefficinet_derivative_y[i] = 0
        if abs(coefficinet_derivative_z[i]) < tol or abs(dc[i])>rc:
            coefficinet_derivative_z[i] = 0
    
    coefficinet_derivative = np.array(
        [coefficinet_derivative_x, coefficinet_derivative_y, coefficinet_derivative_z]).T
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    area_voronoi = m.diagonal()
    First_Term = coef[:, None]*AG
    Second_Term =area_voronoi[:, None]*coefficinet_derivative
    Fad = -(First_Term + Second_Term)
    Ead=-(coef*area_voronoi)
    total_EAD=np.sum(Ead)
    return Fad,total_EAD

In [10]:
####Random Forces
KbT=0.01
dt=0.01
sigma=np.sqrt(2*0.001*(KbT/dt))

def fun_FR(v,f): 
    edges, fe, ef = igl.edge_topology(v, f)
    Fr=np.zeros((len(v),3))
    for e in range(len(edges)): 
        rij=v[edges[e][0]]-v[edges[e][1]]
        rij_norm = rij/np.linalg.norm(rij)
        gaussian=np.random.normal(0, 1, 1)
        Force=gaussian*rij_norm*sigma
        Fr[edges[e][0]]+=Force
        Fr[edges[e][1]]-=Force  
    return Fr

In [11]:
def Total_Force(FB,FA,FV,Fad,FR):
    Total_force=(FB+FA+FV+0*Fad+0*FR)
    return Total_force

In [12]:
def updateTimeStep(v,f,l,TF,initialMaxForce,dt_size2_ratio,charTimeStep):
    currentMinSize = np.amin(l)
    currentMaxForce = np.amax(np.linalg.norm(TF, axis=1))
    dt = (dt_size2_ratio * currentMinSize **2)*\
        (initialMaxForce / currentMaxForce)
    
    if (charTimeStep / dt > 1e3):
        print("Time step too small! May consider restarting\n",
              "simulation in small time scale")
        print("Current size / initial size =",
              currentMinSize /np.sqrt(charTimeStep/dt_size2_ratio))
        print("Current forece / inital force =",
             currentMaxForce / initialMaxForce)
        exit()
    return dt

In [ ]:
### main loop for simulation
l = igl.edge_lengths(v_init,f)
l0 = igl.avg_edge_length(v_init,f)
dt_size2_ratio = charTimeStep / np.amin(l)**2
number_of_vertices=len(v_init)
###Forward Euler Main 
#Setup

v=v_init
vel=np.zeros((len(v_init),3))
time=0
#FD=fun_FD2(v_init,f,vel)

#Volume_old=cal_volume(v,f)
#Area_t=cal_Area(v,f)
grad_Area=areaGrad(v_init,f)
grad_Volume=volGrad(v_init,f)
Area_new=cal_areatot(v_init,f)
print('initial area = ',Area_new)
Volume_new=cal_volumetot(v_init,f)
print('initial volume = ', Volume_new)

FB,EB=Force_Bending(v_init,f)
FA=Force_Area(Area_t,grad_Area,Area_new)
FV=Force_Volume(Volume_t,grad_Volume,Volume_new)
Fad,EAD=Force_Adhersive(v_init,f,x,y,z,rho,U,grad_Area)
FR=fun_FR(v,f)
#FR=0*Force_Random(v,f,charTimeStep)
TF=Total_Force(FB,FA,FV,Fad,FR)
initialMaxForce = np.amax(np.linalg.norm(TF, axis=1))

### Calculation of Energy
timeout=[]
totalEnergy=[]
EnergyArea=[]
EnergyVolume=[]
EnergyBending=[]
EnergyAdhesive=[]

for i in range(iterations):
    #Integration
    
    vel = TF/gamma
    
    ## adjust time step if adopt adaptive time step based on mesh size
    if (isAdaptiveStep):
        charTimeStep = updateTimeStep(v,f,l,TF,initialMaxForce,
                                                dt_size2_ratio,charTimeStep);
        
    dt = charTimeStep
    
    v += vel*dt
    time += dt
    
    ###Mesh Regularization
    l = igl.edge_lengths(v,f)
    v=ver_new(v,f)
    if (not igl.is_intrinsic_delaunay(l,f).all()):
        #v,f,_=pymesh.split_long_edges_raw(v,f,l0*4/3)
        print('time =',time,': not all edges are delaunay, call intrinsic delaunay triangulation')
        l, f= igl.intrinsic_delaunay_triangulation(l,f)
   
    #Force calculation
    Area_current=cal_areatot(v,f)
    grad_Area_new=areaGrad(v,f)
    Volume_current=cal_volumetot(v,f)
    grad_Volume_new=volGrad(v,f)
    ###Update forces here
 
    FB,EB=Force_Bending(v,f) ##bending_force and Bending Energy
    FA=Force_Area(Area_t,grad_Area_new,Area_current)
    FV=Force_Volume(Volume_t,grad_Volume_new,Volume_current)
    Fad,EAD=Force_Adhersive(v,f,x,y,z,rho,U,grad_Area_new)
    FR=fun_FR(v,f)

    TF=Total_Force(FB,FA,FV,Fad,FR)

    
    mechErrorNorm = np.sum(np.linalg.norm(TF, axis=1))

    if i%outfrequency==0:
        print('time =',time)
        print('dt =',dt)
        print('dVolume/Volume_t =',Volume_current-Volume_t,'/',Volume_t)
        print('dArea/Area_t =',Area_current-Area_t,'/',Area_t)
        print('mechErrorNorm =',mechErrorNorm)
        timeout.append(time)
        EnergyArea.append(Energy_area(v,f,Area_t))
        EnergyVolume.append(Energy_volume(v,f,Volume_t))
        EnergyBending.append(EB)
        EnergyAdhesive.append(EAD)
        PotentialEnergy = np.array(EnergyArea) + np.array(EnergyVolume) + np.array(EnergyBending)+np.array(EnergyAdhesive)
        
        print('PE = ',PotentialEnergy[-1])
        
#         istr=str(i)
#         filename='dynamics'+istr+'.png'
#         fig = plt.figure()

#         fig = plt.figure(figsize =(14, 8))
#         ax = plt.axes(projection ='3d')
#         surf=ax.plot_trisurf(v[:,0], v[:,1], v[:,2], triangles = f, edgecolor=[[0,0,0]], linewidth=0.4, alpha=0.0, shade=False)
#         #ax.plot_surface(x, y, z)
#         u, v1 = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
#         x1 = (rp*(np.cos(u) * np.sin(v1)))+x
#         y1 = (rp*(np.sin(u) * np.sin(v1)))
#         z1 = (rp*(np.cos(v1)))
#         surf=ax.plot_surface(x1, y1, z1)
#         ax.view_init(90,180)
#         plt.savefig(os.path.join(root_folder,'Dynamics/',filename))
#         mp.jupyter()
#         p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
#                                "width": 300, "height": 300},return_plot=True)
#         #p.add_lines(v,v+1*FB, shading={"line_color": "red"})
#         name = 'test'+str(i)
#         print('number of vertices=',len(v))
        print('number of Iterations=',i)
    if i>=10 and i%outfrequency==0:
        EnergyChangeRate=(abs(PotentialEnergy[-1]-PotentialEnergy[-2])/(PotentialEnergy[-1]))*100
        print('Energy_Change_Rate =', EnergyChangeRate)
    if i>=100 and EnergyChangeRate<tolerance:
        print('Change of Energy is very small \n Reached Equilibrioum Shape')
#         igl.write_triangle_mesh(os.path.join(root_folder,'08_23_2022/',filename), v, f)
#         np.savetxt(os.path.join('08_23_22',"PE_u=4,rho=5.txt"), (PotentialEnergy))
#         np.savetxt(os.path.join('08_23_22',"BE_u=4,rho=5.txt"), (EnergyBending))
        print('Storing Off file')
        break

initial area =  12.543485314563185
initial volume =  4.174596635472643
time = 0.1 : not all edges are delaunay, call intrinsic delaunay triangulation
time = 0.1
dt = 0.1
dVolume/Volume_t = 0.3897144632658214 / 3.7680000000000002
dArea/Area_t = -0.025343072604027128 / 12.56
mechErrorNorm = 87.52827357081318
PE =  0.6413369463661005
number of Iterations= 0
time = 0.10208669985026159 : not all edges are delaunay, call intrinsic delaunay triangulation
time = 0.10689368241015434 : not all edges are delaunay, call intrinsic delaunay triangulation
time = 0.10999480774560484 : not all edges are delaunay, call intrinsic delaunay triangulation
time = 0.3517972734459593
dt = 0.006324950944611173
dVolume/Volume_t = 0.35260589914587337 / 3.7680000000000002
dArea/Area_t = 0.8837859819559544 / 12.56
mechErrorNorm = 718.4698773019256
PE =  7.148985966699526
number of Iterations= 50
Energy_Change_Rate = 91.02898020287783
time = 0.6212238517003122
dt = 0.006032999666272485
dVolume/Volume_t = 0.322383409

time = 7.5111138585070325
dt = 0.004905648197179098
dVolume/Volume_t = 0.15564220971653597 / 3.7680000000000002
dArea/Area_t = 0.21012748827893546 / 12.56
mechErrorNorm = 604.5939240413653
PE =  4.306806085561485
number of Iterations= 1350
Energy_Change_Rate = 8.568246450704388
time = 7.7709735430261455
dt = 0.004359920976474258
dVolume/Volume_t = 0.15379500699277227 / 3.7680000000000002
dArea/Area_t = -0.02875150227808909 / 12.56
mechErrorNorm = 478.1081396658451
PE =  2.5015223942976497
number of Iterations= 1400
Energy_Change_Rate = 72.16740075479929
time = 8.031181362968248
dt = 0.004438035607256971
dVolume/Volume_t = 0.15130742070745828 / 3.7680000000000002
dArea/Area_t = 0.026244110139407084 / 12.56
mechErrorNorm = 506.8506783437402
PE =  2.8430864700167735
number of Iterations= 1450
Energy_Change_Rate = 12.013847602641107
time = 8.291084890664168
dt = 0.004939799303383551
dVolume/Volume_t = 0.15393802772038212 / 3.7680000000000002
dArea/Area_t = 0.2462401604260851 / 12.56
mechEr

time = 15.14744259215699
dt = 0.005317669819296011
dVolume/Volume_t = 0.15355638095855406 / 3.7680000000000002
dArea/Area_t = 0.45497746168681097 / 12.56
mechErrorNorm = 610.8664908143671
PE =  4.7468257745741385
number of Iterations= 2850
Energy_Change_Rate = 53.98798907278158
time = 15.396764484556192
dt = 0.004213230473826895
dVolume/Volume_t = 0.15359351533280297 / 3.7680000000000002
dArea/Area_t = 0.09396046829015425 / 12.56
mechErrorNorm = 445.42186576967737
PE =  2.3004491700982044
number of Iterations= 2900
Energy_Change_Rate = 106.34343224247375
time = 15.648095676942319
dt = 0.005426663002102337
dVolume/Volume_t = 0.1503388775235206 / 3.7680000000000002
dArea/Area_t = 0.4701925604746826 / 12.56
mechErrorNorm = 611.0923944338057
PE =  4.802142230063847
number of Iterations= 2950
Energy_Change_Rate = 52.09535536668978
time = 15.898335864833003
dt = 0.0045008206010766156
dVolume/Volume_t = 0.15032992212211438 / 3.7680000000000002
dArea/Area_t = 0.20114016945785274 / 12.56
mechEr

time = 22.631940085552603
dt = 0.005117666705164479
dVolume/Volume_t = 0.1185947665362721 / 3.7680000000000002
dArea/Area_t = 0.4376309870116746 / 12.56
mechErrorNorm = 575.2193235328257
PE =  4.231356731564375
number of Iterations= 4350
Energy_Change_Rate = 36.99617646492811
time = 22.88166071717301
dt = 0.004431592211148784
dVolume/Volume_t = 0.11087040887131838 / 3.7680000000000002
dArea/Area_t = 0.23305168256214337 / 12.56
mechErrorNorm = 490.18433301931134
PE =  2.798504553963312
number of Iterations= 4400
Energy_Change_Rate = 51.2006376967235
time = 23.131186710186718
dt = 0.0051407627930399715
dVolume/Volume_t = 0.11447520149773904 / 3.7680000000000002
dArea/Area_t = 0.44780557539710486 / 12.56
mechErrorNorm = 577.8740861647668
PE =  4.2725999001906985
number of Iterations= 4450
Energy_Change_Rate = 34.50113234711243
time = 23.380623481000775
dt = 0.004406546650125333
dVolume/Volume_t = 0.10599685963815064 / 3.7680000000000002
dArea/Area_t = 0.22780181416685252 / 12.56
mechError

In [ ]:
time=np.linspace(0,i,len(EnergyBending))
# plt.plot(time,totalEnergy,'o-', color='red', label='total Energy')
plt.plot(time,EnergyBending,'o-', label='Bending Energy')
plt.plot(time,EnergyVolume,'v-', label='Volume Energy')
plt.plot(time,EnergyArea,'o-', label='Area Energy')
plt.plot(time,PotentialEnergy,'x-',label='Potential Energy')
plt.plot(time,EnergyAdhesive,'<-',label='Adhesive Energy')
# plt.plot(time,KineticEnergy,'v-', label='Kinetic Energy',color='k')
#plt.axis([0, 800, 0, 2])
plt.legend(loc='lower right')
plt.title('timesteps vs Energy')
plt.show()

In [ ]:
# fig = plt.figure()

# fig = plt.figure(figsize =(14, 8))
# ax = plt.axes(projection ='3d')
# surf=ax.plot_trisurf(v[:,0], v[:,1], v[:,2], triangles = f, edgecolor=[[0,0,0]], linewidth=0.4, alpha=0.0, shade=False)
# #ax.plot_surface(x, y, z)
# rp =0.161
# u, v1 = np.mgrid[0:2 * np.pi:30j, 0:np.pi:20j]
# x = (rp*(np.cos(u) * np.sin(v1)))+1.161
# y = (rp*(np.sin(u) * np.sin(v1)))
# z = (rp*(np.cos(v1)))
# surf=ax.plot_surface(x, y, z)
# ax.view_init(90,180)
# filename= 'test'+'.png'
# plt.savefig(os.path.join(root_folder,'Dynamics/',filename))

In [ ]:
mp.jupyter()
p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                       "width": 300, "height": 300},return_plot=True)
#p.save("u=2.4,rho=1.html")
#p.add_lines(v,v+1*FB, shading={"line_color": "red"})


distance=np.zeros(len(v))
coef=np.zeros(len(v))
dc=np.zeros(len(v))
for i in range(len(v)):
    distance[i]=(np.sqrt((((v[i][0])-x)**2)+(((v[i][1])-y)**2)+(((v[i][2])-z)**2)))
    dc[i]=distance[i]-(rp)
type(dc)
print("Min distance:",dc.min())
print("Max distance:",dc.max())
def histogram_distances_default(dc):
    hist, bin_edges = np.histogram(dc)
    return hist, bin_edges

def plot_histogram(hist,bin_edges):
    #for N bins, there are N+1 bin edges. The centers can be found by averaging the positions of 
    # bin edge0 and 1, 1 and 2, ..., N-1 and N
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2.0
    plt.plot(bin_centers,hist,marker='')
    plt.ylabel("Number of Vertices(r)")
    plt.xlabel("radial distance")
    plt.title("Radial Distribution Functio")

dist_hist_1, bin_edges_1 = histogram_distances_default(dc)
plot_histogram(dist_hist_1,bin_edges_1)
